## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-03-19-11-55 +0000,nypost,House narrowly approves $1.2T funding deal to ...,https://nypost.com/2026/02/03/us-news/house-na...
1,2026-02-03-19-09-16 +0000,nyt,"Michael Parenti, Unapologetic Marxist Theorist...",https://www.nytimes.com/2026/02/03/us/politics...
2,2026-02-03-19-08-43 +0000,nyt,What to Know About the Disappearance of Savann...,https://www.nytimes.com/2026/02/03/us/savannah...
3,2026-02-03-19-05-00 +0000,wsj,The Famous Names in the Latest Release of Epst...,https://www.wsj.com/us-news/law/what-we-know-s...
4,2026-02-03-19-04-43 +0000,bbc,Disney names Josh D'Amaro as new chief executive,https://www.bbc.com/news/articles/cy4qxdq2147o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2490/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
90,trump,44
24,epstein,26
0,house,17
4,deal,14
29,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
151,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...,131
321,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...,125
25,2026-02-03-18-28-00 +0000,wsj,Bill and Hillary Clinton reached a deal to app...,https://www.wsj.com/politics/clintons-reach-de...,110
64,2026-02-03-17-40-00 +0000,wsj,The GOP-led House narrowly cleared a critical ...,https://www.wsj.com/politics/policy/government...,90
257,2026-02-03-03-17-41 +0000,nyt,Trump Presses House G.O.P. to Back Deal to Reo...,https://www.nytimes.com/2026/02/02/us/politics...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
151,131,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...
321,108,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...
7,64,2026-02-03-19-02-45 +0000,nyt,"Authorities Believe Savannah Guthrie’s Mother,...",https://www.nytimes.com/2026/02/03/us/savannah...
101,59,2026-02-03-16-19-18 +0000,bbc,Son of Norway's crown princess hears details o...,https://www.bbc.com/news/articles/ckgn8dnxjgwo...
169,51,2026-02-03-13-32-11 +0000,latimes,Disney names theme parks head Josh D'Amaro as ...,https://www.latimes.com/entertainment-arts/bus...
64,47,2026-02-03-17-40-00 +0000,wsj,The GOP-led House narrowly cleared a critical ...,https://www.wsj.com/politics/policy/government...
314,46,2026-02-02-22-57-52 +0000,cbc,U.S. judge grants asylum to Chinese whistleblo...,https://www.cbc.ca/radio/asithappens/china-whi...
308,38,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...
379,36,2026-02-02-19-18-09 +0000,nypost,Prince Andrew’s ex Fergie once begged Jeffrey ...,https://nypost.com/2026/02/02/world-news/princ...
377,35,2026-02-02-19-28-14 +0000,bbc,Gold and silver prices fall but FTSE 100 hits ...,https://www.bbc.com/news/articles/cpdy0y3jle4o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
